In [1]:
import gurobipy as gp
from gurobipy import GRB

In [8]:
a= 900
print(a//9)

100


In [ ]:
# amount of each categories for company bundle
Company, paper, glass, plastic  = gp.multidict({
    'A' : [5,3,1],
    'B' : [7,3,2],
    'C' : [2,1,1],
    'D' : [3,1,4]
})

# amount of supply, already subtracted from the company relationship require
Categories, amount = gp.multidict({
    'paper' : 13300,
    'glass' : 6200,
    'plastic' : 13200
})

# profit from company bundle
Company, profit_bundle = gp.multidict({
    'A' : 2, # per 9lbs bundle
    'B' : 3, # per 10lbs bundle
    'C' : 1, # per 4lbs bundle
    'D' : 2 # per 9lbs bundle
})

# profit from small recycler
Categories, profit_single = gp.multidict({
    'paper' : 0.05,
    'glass' : 0.03,
    'plastic' : 0.02
})


1) 

Total collection

15,000 pounds of paper

7,000 pounds of glass

14,000 pounds of plastic

In [ ]:
model_revenue1 = gp.Model(name = 'revenue')

model_revenue1.addVars(Categories, name = 'materials')
model_revenue1.setObjective()

5
